![header](https://capsule-render.vercel.app/api?type=waving&color=gradient&width=1000&height=200&section=header&text=Blockchain%20Python&fontSize=30&fontColor=black)

In [1]:
# Import dependencies
import subprocess, os
import json
from dotenv import load_dotenv

In [2]:
# Load and set environment variables
load_dotenv("resources/mnemonic.env")
mnemonic = os.getenv("mnemonic")
print(f"Mnemonic : {mnemonic}")

Mnemonic : foot animal peace fragile another screen amused glide news people detect ramp


In [3]:
# Import constants.py and necessary functions from bit and web3
from constants import *
from bit.wallet import PrivateKeyTestnet, NetworkAPI
from web3 import Account, Web3, HTTPProvider 

## *Derive deterministic wallets*

In [4]:
# Create a function called `derive_wallets`
def derive_wallets(coin : str,
                   mnemonic : str = mnemonic,
                   numderive : int = 3):
    
    command = f"./derive -g --mnemonic=\"{mnemonic}\" --coin={coin} --numderive={str(numderive)} --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --format=json"
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [5]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {}
coins[BTCTEST] = derive_wallets(mnemonic=mnemonic, coin=BTCTEST)
coins[ETH] = derive_wallets(mnemonic=mnemonic, coin=ETH)

In [6]:
print("1st address in BTC-TEST blockchain derived from mnemonic :")
from pprint import pprint as pprint
pprint(coins[BTCTEST][0])

1st address in BTC-TEST blockchain derived from mnemonic :
{'address': 'mx5943UjQyiEPJZe1AbvfLBk3bc5ydNQAo',
 'index': 0,
 'path': "m/44'/1'/0'/0/0",
 'privkey': 'cPNgiXNwcapfe93SUK5YFn8YRAe8C5LESGT4Grwwhk3r3usMCgie',
 'pubkey': '02647bb6672062db5370dc2281751e15cb0992a3d4aaef85fc8f726eea27ca5b08',
 'pubkeyhash': 'b5984dd3ad20d0a2f1097f0dc19806ac2dddddcb',
 'xprv': 'tprv8koAkJFiHT4jHiAsF8AgWXZi9kNNWuUcApBjyeiBAtSYQHmKvewYAUvfJzhYkLkG4P8W15G1QVw7fqKUjv1dC6dRxCgK5cKAs7GmRmUGwKQ',
 'xpub': 'tpubDHVCtiHxRpkQBBCf8mqGuwDpimtJgEfWk7nXGAkUbAEwEn26Z3m8LyYXV8191Gsez6audjhg5dhP7NyXfgFTyBVmKfCPTuVPHatpMdttmxd'}


In [7]:
print("1st address in ETH blockchain derived from mnemonic :")
from pprint import pprint as pprint
pprint(coins[ETH][0])

1st address in ETH blockchain derived from mnemonic :
{'address': '0xc61803d7eC5247f8d61f0a015B41D2a40b7163b8',
 'index': 0,
 'path': "m/44'/60'/0'/0/0",
 'privkey': '0x83d3e09966d875b7c6f2c78dc896973fdf725ea098e91f5b25961c4f00d97de7',
 'pubkey': '0262718bf3764bf7f7fb359f97aa6214b9f5888f78e567574350097681b59c4c75',
 'pubkeyhash': 'ce0ff31ea47bf38d5869307f86c143e2e1e750b0',
 'xprv': 'xprvA2m1CSP2VoQHnmNaQdEcabhhKJ2pv8vG7AY2YoKfn6p8SJqtx785gXjvCiQG192S5xxWVrTirjsmNtkw6QRRE1tu2aqrSCwKnDeRTJAKC5u',
 'xpub': 'xpub6FkMbwuvLAxb1FT3WemcwjeRsKsKKbe7UPTdMBjHLSM7K7B3VeSLEL4Q3yBKh9tmQ7zBbfnbPvCG18EptF6xumoXYDv1cKNnzjRPQkvigEE'}


In [ ]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin : str,
                       priv_key : str):
    '''Converts the privkey string in a child key to an account object 
    that bit or web3.py can use to transact'''
        
    if coin == "btc-test":
        return PrivateKeyTestnet(priv_key)
    
    elif coin == "eth":
        return Account.privateKeyToAccount(priv_key)


In [ ]:
eth_acc_1 = priv_key_to_account(ETH, coins[ETH][0]['privkey'])
print(f"1st Ethereum public key : {eth_acc_1.address}")

eth_acc_2 = priv_key_to_account(ETH, coins[ETH][1]['privkey'])
print(f"2nd Ethereum public key : {eth_acc_2.address}")

In [ ]:
btc_acc_1 = priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey'])
print(f"1st Bitcoin-Testnet public key : {btc_acc_1.address}")

btc_acc_2 = priv_key_to_account(BTCTEST, coins[BTCTEST][1]['privkey'])
print(f"2nd Bitcoin-Testnet public key : {btc_acc_2.address}")

## *Connect to blockchain nodes*

In [ ]:
# Connect to Etherum Node
url = "http://127.0.0.1:8545"
w3 = Web3(HTTPProvider(url))

from web3.middleware import geth_poa_middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

# Check connection
print(f"Connected to Ethereum node {url}: {w3.isConnected()}")
print(f"ChainId : {w3.eth.chainId}")

## *Create transaction*

In [ ]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(acc, 
              coin : str,
              to : str,
              amount : float or int,
              gas : int = 1):
    '''Creates the raw, unsigned transaction that contains all metadata 
        needed to transact'''
    
    if coin == "btc-test":
        return PrivateKeyTestnet.prepare_transaction(acc.address, [(to, amount*100000000, 'usd')])
    
    elif coin == "eth":
        return {'to': to,
                'from' : str(acc.address),
                'value': w3.toWei(amount,'ether'),
                'gas': int(w3.eth.gasPrice / 100000),
                'gasPrice': w3.eth.gasPrice,
                'nonce': w3.eth.getTransactionCount(acc.address),
                'chainId': w3.eth.chainId
                }
        

In [ ]:
# Test Ethereum transaction
eth_transaction = create_tx(acc = eth_acc_1, 
                            coin=ETH, 
                            to="0xc61803d7eC5247f8d61f0a015B41D2a40b7163b8", 
                            amount=1)
pprint(eth_transaction)

In [ ]:
# Test Bitcoin transaction
btc_transaction = create_tx(acc = btc_acc_1, 
                            coin=BTCTEST, 
                            to="mx5943UjQyiEPJZe1AbvfLBk3bc5ydNQAo", 
                            amount=0.0000001)
pprint(btc_transaction)

## *Send transaction*

In [ ]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(account, 
            coin : str,
            to : str,
            amount : float or int,
            gas : int = 1):
    
    raw_tx = create_tx(acc=account,
                       coin=coin,
                       to=to,
                       amount=amount,
                       gas=gas)
    
    if coin == "btc-test":
        # Sign the transaction with your new wallet private key
        signed = account.sign_transaction(raw_tx)
        
        # Transmit the transaction
        NetworkAPI.broadcast_tx_testnet(signed)
        print(f"{amount} tBTC sent from : {account.address} to : {to}")
    
    elif coin == "eth":
        # Sign the transaction with your new wallet private key
        signed = account.sign_transaction(raw_tx)
        
        # Transmit the transaction
        w3.eth.sendRawTransaction(signed.rawTransaction)
        print(f"{w3.fromWei(amount, 'ether')} eth sent from : {account.address} to : {to}") 
        print(f"Signed transaction : {signed.rawTransaction.hex()}")
        print(f"Signed transaction hash : {signed.hash.hex()}")

In [ ]:
# Test Bitcoin transaction
btc_acc = priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey'])
btc_transaction = send_tx(account = btc_acc, 
                          coin=BTCTEST,
                          to="mkQwyRXQY19EZvY67JhdMpkzzQ8SkiiGUQ",
                          amount=0.00000000001)

In [ ]:
# Test Ethereum transaction
eth_acc = priv_key_to_account(ETH, coins[ETH][0]['privkey'])
eth_transaction = send_tx(account = eth_acc, 
                          coin=ETH,
                          to="0x28E72008368b60bf3400232786aeDc603990F15F",
                          amount=10)